In [47]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Accuracy


# 整理数据
# 将三个文件的数据整合到data中

# 导入三个数据集
happy = np.load(f'./process/happy.npy')[:8200]
normal = np.load(f'./process/normal.npy')[:8200]
sad = np.load(f'./process/sad.npy')[:8200]

In [49]:
# 预处理
happy = happy.reshape(-1, 50)
normal = normal.reshape(-1, 50)
sad = sad.reshape(-1, 50)
happy = np.stack([happy,happy,happy])
normal = np.stack([normal,normal,normal])
sad = np.stack([sad,sad,sad])

happy_label = np.full((happy.shape[0],1),0)
normal_label = np.full((normal.shape[0],1),1)
sad_label = np.full((sad.shape[0],1),2)

In [50]:
# 检查数据维度
print(happy.shape, normal.shape, sad.shape)
print(happy_label.shape,normal_label.shape,sad_label.shape)


(3, 164, 50) (3, 164, 50) (3, 164, 50)
(3, 1) (3, 1) (3, 1)


In [51]:
# 定义模型
# 输入层
input = tf.keras.layers.Input((164,50))

# 卷积层1
conv1 = tf.keras.layers.Conv1D(32,16,activation='relu',padding='same')(input)
drop1 = tf.keras.layers.Dropout(0.2)(conv1)
max1 = tf.keras.layers.Dense(8)(drop1)

# f = tf.keras.layers.Flatten（
dense1 = tf.keras.layers.Dense(128,'relu')(max1)
dense2 = tf.keras.layers.Dense(64, 'relu')(dense1)
output = tf.keras.layers.Dense(3, 'softmax')(dense2)

#
np.set_printoptions(suppress=True)

# 定义模型
model = tf.keras.Model(inputs=input,outputs=output)

adam = tf.keras.optimizers.Adam(0.0002)

model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 164, 50)]         0         
                                                                 
 conv1d_2 (Conv1D)           (None, 164, 32)           25632     
                                                                 
 dropout_1 (Dropout)         (None, 164, 32)           0         
                                                                 
 dense_4 (Dense)             (None, 164, 8)            264       
                                                                 
 dense_5 (Dense)             (None, 164, 128)          1152      
                                                                 
 dense_6 (Dense)             (None, 164, 64)           8256      
                                                                 
 dense_7 (Dense)             (None, 164, 3)            195 

In [52]:
model.fit(happy,happy_label,
          epochs=1,    
          verbose=2)

z = model.predict(happy)

InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "e:\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "e:\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "e:\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "e:\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "e:\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "e:\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "e:\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "e:\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispatch_queue

  File "e:\anaconda3\Lib\sit

In [255]:
z

array([[[0.8523192 , 0.03136802, 0.1163128 ],
        [0.99466044, 0.00390266, 0.00143684],
        [0.94598895, 0.0539976 , 0.00001344],
        ...,
        [0.5357061 , 0.35970923, 0.10458466],
        [0.9202194 , 0.07031878, 0.00946185],
        [0.98358685, 0.01509249, 0.00132059]],

       [[0.87416995, 0.02456729, 0.10126275],
        [0.99667287, 0.00248579, 0.00084134],
        [0.9567057 , 0.04328886, 0.00000543],
        ...,
        [0.54882425, 0.35743308, 0.09374262],
        [0.9354365 , 0.05795937, 0.00660413],
        [0.9884537 , 0.01077626, 0.00077009]],

       [[0.8698677 , 0.02586203, 0.10427027],
        [0.9963277 , 0.00273143, 0.00094086],
        [0.9546509 , 0.04534248, 0.00000657],
        ...,
        [0.5461276 , 0.35795125, 0.09592103],
        [0.93251556, 0.06036339, 0.00712096],
        [0.98757523, 0.0115628 , 0.00086197]]], dtype=float32)